# Deep learning - project 1
## Creating and training neural network from scratch

In [103]:
import numpy as np
from typing import List, Callable, Iterable
from abc import ABC, abstractmethod 

In [250]:
class Layer:
    """
    One layer of a neural network. Used only when creating neural networks.
    # attributes: number of neurons
    # implements: operator() which depends on activation function, derivative()
    """
    def __init__(self, n_neurons: int, activation: Callable[[np.array], np.array], *, verbose=False):
        self.n_neurons = n_neurons
        self.output_size = n_neurons
        self.activation = activation
        self.verbose = verbose
        
    def verbose_print(self, s: str):
        if self.verbose:
            print(s)
    
    def initalize(self, input_size, *, sd=0.05):
        self.input_size = input_size
        self.W = np.random.normal(0, sd, size=[input_size, self.output_size])  # weights
        self.b = np.random.normal(0, sd, size=[1, self.output_size])  # biases
    
    def __call__(self, input_vector: np.array):
        self.verbose_print("calculating next vector from sizes {}x{} with matrix {}x{}".format(*input_vector.shape, self.input_size, self.output_size))
        return self.activation(input_vector.dot(self.W) + self.b)
    
    def __str__(self):
        return "Layer(input_size={}, output_size={})".format(self.input_size, self.output_size)
    
    def __repr__(self):
        return str(self)
    

class Loss(ABC):
    """
    Base class for a loss function of a network
    # implements: operator(), derivative()
    """
    @abstractmethod
    def __call__(self, y_predicted: np.array, y_true: np.array):
        pass

In [251]:
class QuadraticLoss(Loss):
    """Loss for simple regression: mean squared error."""
    def __call__(self, y_predicted: np.array, y_true: np.array):
        if len(y_predicted) != len(y_true):
            raise IndexError("length of y_predicted ({}) has to be the same as lenght of y_true ({})".format(len(y_predicted), len(y_true)))
        return np.linalg.norm(y_predicted - y_true) / len(y_true)
    
class BernLoss(Loss):
    """
    Loss for binary classification (negative binomial likelihood),
    also known as cross-entropy between the between empirical and model distribution (binomial).
    """
    def __call__(self, y_predicted: np.array, y_true: np.array):
        return -np.mean(
            np.array(
                [np.log(p) if y == 1 else np.log(1-p) 
                 for p, y in zip(y_predicted, y_true)]
            )
        )
    
def sigmoid(x: np.array) -> np.array:
    return 1 / (1 +  np.exp(-x))

def identity(x: np.array) -> np.array:
    return x


In [252]:
class NNet:
    """Feedforwad (classical) neural network"""
    def __init__(self, input_size: int, layers: List[Layer], loss: Loss):
        layers[0].initalize(input_size)
        for i, layer in enumerate(layers):
            if i == 0:
                continue
            layer.initalize(input_size=layers[i-1].output_size)
        self.layers = layers
        self.loss = loss
        
    def __call__(self, x: np.array):
        """Forwad pass"""
        y = x
        for layer in self.layers:
            y = layer(y)
        return y
        

In [253]:
nnet = NNet(
    10,
    [Layer(5, sigmoid), Layer(11, sigmoid), Layer(1, identity)],
    QuadraticLoss()
)

In [254]:
x = np.ones([100, 10])

In [255]:
nnet(x)

array([[0.01067473],
       [0.01067473],
       [0.01067473],
       [0.01067473],
       [0.01067473],
       [0.01067473],
       [0.01067473],
       [0.01067473],
       [0.01067473],
       [0.01067473],
       [0.01067473],
       [0.01067473],
       [0.01067473],
       [0.01067473],
       [0.01067473],
       [0.01067473],
       [0.01067473],
       [0.01067473],
       [0.01067473],
       [0.01067473],
       [0.01067473],
       [0.01067473],
       [0.01067473],
       [0.01067473],
       [0.01067473],
       [0.01067473],
       [0.01067473],
       [0.01067473],
       [0.01067473],
       [0.01067473],
       [0.01067473],
       [0.01067473],
       [0.01067473],
       [0.01067473],
       [0.01067473],
       [0.01067473],
       [0.01067473],
       [0.01067473],
       [0.01067473],
       [0.01067473],
       [0.01067473],
       [0.01067473],
       [0.01067473],
       [0.01067473],
       [0.01067473],
       [0.01067473],
       [0.01067473],
       [0.010